## Module 10 Homework 
### Please complete the following questions.

1. Import the "babies.xlsx" dataset. See below for information on the columns:

    * bwt - birth weight of newborn baby
    * gestation	- gestation length (weeks)
    * parity - previously pregnant (0 = no; 1 = yes)
    * age - age of mother
    * height - height of mother (inches)	
    * weight - weight of mother (pounds)
    * smoke - smoking status of mother (0 = nonsmoker; 1 = smoker)

In [1]:
import pandas as pd
import numpy as np
from scipy import stats


In [2]:
df = pd.read_excel("babies.xlsx")

2. Preview the first few rows of the dataset. Complete the following checks for your dataset:

    * output the summary information for the dataset, what types of variables make up this dataset?
    * is there any missing data in this dataset?

In [3]:
df.head()

bwt  gestation  parity   age  height  weight  smoke
0  120      284.0       0  27.0    62.0   100.0    0.0
1  113      282.0       0  33.0    64.0   135.0    0.0
2  128      279.0       0  28.0    64.0    77.0    1.0
3  123        NaN       0  36.0    69.0   190.0    0.0
4  108      282.0       0  23.0    67.0   125.0    1.0

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1236 entries, 0 to 1235
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   bwt        1236 non-null   int64  
 1   gestation  1223 non-null   float64
 2   parity     1236 non-null   int64  
 3   age        1234 non-null   float64
 4   height     1214 non-null   float64
 5   weight     1200 non-null   float64
 6   smoke      1226 non-null   float64
dtypes: float64(5), int64(2)
memory usage: 67.7 KB


In [5]:
# 1236 rows 7 columns
# data is int64(2) and float64(5) numbers

df.isnull().sum()
# there are missing data in gestaition, age, hieghts, wieght, smoke


bwt           0
gestation    13
parity        0
age           2
height       22
weight       36
smoke        10
dtype: int64

3. Handle the missing data in the dataset -- there isn't much, so we can drop all the rows that have at least one missing value. How many rows were dropped from the dataset?

In [6]:
df.loc[df.isnull().any(axis = 1)] # 62 rows contain missing data

bwt  gestation  parity   age  height  weight  smoke
3     123        NaN       0  36.0    69.0   190.0    0.0
39    128      282.0       0  31.0    65.0     NaN    0.0
42    138      302.0       0  26.0     NaN     NaN    1.0
85    129      274.0       0  29.0    71.0     NaN    1.0
89    114        NaN       0  24.0    67.0   113.0    1.0
...   ...        ...     ...   ...     ...     ...    ...
1044  117      323.0       0  26.0    62.0     NaN    0.0
1177  146      305.0       0  23.0     NaN     NaN    0.0
1190  104      275.0       0  24.0     NaN     NaN    0.0
1192  124        NaN       1  39.0    65.0   228.0    0.0
1215  152      267.0       0  28.0     NaN   119.0    1.0

[62 rows x 7 columns]

In [7]:
df.dropna(inplace = True)

In [8]:
df.isna().sum()

bwt          0
gestation    0
parity       0
age          0
height       0
weight       0
smoke        0
dtype: int64

4. There are two qualitative variables in this dataset. What are they? How do you know they are qualitative variables?

Smoke and parity are qualitative variables.  The variables are either 1 or 0 to indicate yes or no, meaning they are categories and not qauntitative values.

5. Replace the values in the qualitative variables with meaningful labels that describe the different groups. Use any method you've learned in previous modules to complete this task. 

In [9]:
df["parity"].replace([0,1],["No","Yes"],inplace = True)
df["smoke"].replace([0,1],["nonsmoker","smoker"],inplace = True)

In [10]:
df.head()

bwt  gestation parity   age  height  weight      smoke
0  120      284.0     No  27.0    62.0   100.0  nonsmoker
1  113      282.0     No  33.0    64.0   135.0  nonsmoker
2  128      279.0     No  28.0    64.0    77.0     smoker
4  108      282.0     No  23.0    67.0   125.0     smoker
5  136      286.0     No  25.0    62.0    93.0  nonsmoker

6. Before we move forward with any statistics, let's identify and remove any outliers from the dataset. Using the IQR method, search for outliers in the 5 numeric variables. This will take some time and organization, be careful with your code! Make sure you keep track of how many rows/outliers are removed. 

In [11]:
# create copy
dfq = df.copy()
dfq2 = df.copy()
print(dfq.shape)
print(dfq2.shape)

(1174, 7)
(1174, 7)


In [12]:
# method 1
# drop outlying numeric vars (bwt,gestation,age,height,weight) using inter qurtile range 1.5 rule

# quartiles and IQR
bwt_q1 = dfq["bwt"].quantile(.25)
bwt_q3 = dfq["bwt"].quantile(.75)
bwt_iqr = bwt_q3 - bwt_q1

gest_q1 = dfq["gestation"].quantile(.25)
gest_q3 = dfq["gestation"].quantile(.75)
gest_iqr = gest_q3 - gest_q1

age_q1 = dfq["age"].quantile(.25)
age_q3 = dfq["age"].quantile(.75)
age_iqr = age_q3 - age_q1

ht_q1 = dfq["height"].quantile(.25)
ht_q3 = dfq["height"].quantile(.75)
ht_iqr = ht_q3 - ht_q1

wt_q1 = dfq["weight"].quantile(.25)
wt_q3 = dfq["weight"].quantile(.75)
wt_iqr = wt_q3 - wt_q1


In [13]:
# upper and lower limit
# 1.5 rule below q1 - (1.5 iqr) above q3 + (1.5iqr)

bwt_top = bwt_q3 + (bwt_iqr * 1.5)
bwt_bottom = bwt_q1 - (bwt_iqr * 1.5)

gest_top = gest_q3 + (gest_iqr * 1.5)
gest_bottom = gest_q1 - (gest_iqr * 1.5)
 
age_top = age_q3 + (age_iqr * 1.5)
age_bottom = age_q1 - (age_iqr * 1.5)
 
ht_top = ht_q3 + (ht_iqr * 1.5)
ht_bottom = ht_q1 - (ht_iqr * 1.5)
 
wt_top = wt_q3 + (wt_iqr * 1.5)
wt_bottom = wt_q1 - (wt_iqr * 1.5)


In [14]:
# outlier indices

# iqr_outliers = dfq.loc[(dfq[] > top) | (dfq[] < bottom)].index
bwt_iqr_outliers = dfq.loc[(dfq['bwt'] > bwt_top) | (dfq['bwt'] < bwt_bottom)].index
# drop outliers
dfq = dfq.drop(bwt_iqr_outliers)

gest_iqr_outliers = dfq.loc[(dfq['gestation'] > gest_top) | (dfq['gestation'] < gest_bottom)].index
dfq = dfq.drop(gest_iqr_outliers)

age_iqr_outliers = dfq.loc[(dfq['age'] > age_top) | (dfq['age'] < age_bottom)].index
dfq = dfq.drop(age_iqr_outliers)

ht_iqr_outliers = dfq.loc[(dfq['height'] > ht_top) | (dfq['height'] < ht_bottom)].index
dfq = dfq.drop(ht_iqr_outliers)

wt_iqr_outliers = dfq.loc[(dfq['weight'] > wt_top) | (dfq['weight'] < wt_bottom)].index
dfq = dfq.drop(wt_iqr_outliers)

In [15]:


print("bwt_Q1:", bwt_q1)
print("bwt_Q3:", bwt_q3)
print("bwt_IQR:", bwt_iqr)
print("bwt_Upper Limit:", bwt_top)
print("bwt_Lower Limit:", bwt_bottom)
print("bwt_INDEX VALUES:", bwt_iqr_outliers)
print()

print("gest_Q1:", gest_q1)
print("gest_Q3:", gest_q3)
print("gest_IQR:", gest_iqr)
print("gest_Upper Limit:", gest_top)
print("gest_Lower Limit:", gest_bottom)
print("gest_INDEX VALUES:", gest_iqr_outliers)
print()

print("age_Q1:", age_q1)
print("age_Q3:", age_q3)
print("age_IQR:", age_iqr)
print("age_Upper Limit:", age_top)
print("age_Lower Limit:", age_bottom)
print("age_INDEX VALUES:", age_iqr_outliers)
print()

print("ht_Q1:", ht_q1)
print("ht_Q3:", ht_q3)
print("ht_IQR:", ht_iqr)
print("ht_Upper Limit:", ht_top)
print("ht_Lower Limit:", ht_bottom)
print("ht_INDEX VALUES:", ht_iqr_outliers)
print()

print("wt_Q1:", wt_q1)
print("wt_Q3:", wt_q3)
print("wt_IQR:", wt_iqr)
print("wt_Upper Limit:", wt_top)
print("wt_Lower Limit:", wt_bottom)
print("wt_INDEX VALUES:", wt_iqr_outliers)
print()

dfq.shape

bwt_Q1: 108.0
bwt_Q3: 131.0
bwt_IQR: 23.0
bwt_Upper Limit: 165.5
bwt_Lower Limit: 73.5
bwt_INDEX VALUES: Int64Index([ 239,  309,  361,  462,  500,  529,  556,  594,  632,  709,  738,
             747,  829,  904,  912,  978, 1021, 1035, 1063, 1065, 1099, 1139,
            1148, 1169],
           dtype='int64')

gest_Q1: 272.0
gest_Q3: 288.0
gest_IQR: 16.0
gest_Upper Limit: 312.0
gest_Lower Limit: 248.0
gest_INDEX VALUES: Int64Index([   6,    7,   10,   59,   63,   66,  119,  129,  155,  188,  192,
             198,  210,  215,  217,  234,  240,  253,  260,  279,  345,  373,
             394,  440,  460,  484,  511,  523,  630,  685,  699,  710,  726,
             746,  761,  769,  778,  784,  828,  830,  833,  869,  927,  952,
             969, 1002, 1020, 1026, 1074, 1134, 1140, 1142, 1146, 1152, 1172,
            1178, 1199, 1206, 1207, 1217, 1219, 1226],
           dtype='int64')

age_Q1: 23.0
age_Q3: 31.0
age_IQR: 8.0
age_Upper Limit: 43.0
age_Lower Limit: 11.0
age_INDEX VALUES: In

(1048, 7)

In [16]:
#method 2
#drop outlying numeric vars (bwt,gestation,age,height,weight) using inter qurtile range 1.5 rule
# columns with quantitative values 
columns = ["bwt","gestation","age","height","weight"]

def iqr_outliers(column,df_q,df_):
    #inter quratile range
    q1 = df[column].quantile(.25)
    q3 = df[column].quantile(.75)
    iqr = q3 - q1
    # outlier range with 1.5 rule
    top =  q3 + (iqr * 1.5)
    bottom = q1 - (iqr * 1.5)
    iqr_outliers = df_q.loc[(df_q[column] > top) | (df_q[column] < bottom)].index
    
    print(f"{column} Q1:", q1)
    print(f"{column} Q3:", q3)
    print(f"{column} IQR:", iqr)
    print(f"{column} Upper Limit:", top)
    print(f"{column} Lower Limit:", bottom)
    print(f"{column} INDEX VALUES:", iqr_outliers)
    print() 
    return iqr_outliers



In [17]:
for column in columns:
    # pass original data frame while modifying dfq2 to calculate outliers from original data
    dfq2.drop(iqr_outliers(column,df_q = dfq2,df_ = df),inplace=True)
    

bwt Q1: 108.0
bwt Q3: 131.0
bwt IQR: 23.0
bwt Upper Limit: 165.5
bwt Lower Limit: 73.5
bwt INDEX VALUES: Int64Index([ 239,  309,  361,  462,  500,  529,  556,  594,  632,  709,  738,
             747,  829,  904,  912,  978, 1021, 1035, 1063, 1065, 1099, 1139,
            1148, 1169],
           dtype='int64')

gestation Q1: 272.0
gestation Q3: 288.0
gestation IQR: 16.0
gestation Upper Limit: 312.0
gestation Lower Limit: 248.0
gestation INDEX VALUES: Int64Index([   6,    7,   10,   59,   63,   66,  119,  129,  155,  188,  192,
             198,  210,  215,  217,  234,  240,  253,  260,  279,  345,  373,
             394,  440,  460,  484,  511,  523,  630,  685,  699,  710,  726,
             746,  761,  769,  778,  784,  828,  830,  833,  869,  927,  952,
             969, 1002, 1020, 1026, 1074, 1134, 1140, 1142, 1146, 1152, 1172,
            1178, 1199, 1206, 1207, 1217, 1219, 1226],
           dtype='int64')

age Q1: 23.0
age Q3: 31.0
age IQR: 8.0
age Upper Limit: 43.0
age Lower Li

In [18]:
# method 1 and 2 removes the same number of rows - 126 rows removed.
print(f"original dataframe shape {df.shape}") 
print(f"dfq dataframe shape {dfq.shape}")
print(f"dfq2 dataframe shape {dfq2.shape}")


original dataframe shape (1174, 7)
dfq dataframe shape (1048, 7)
dfq2 dataframe shape (1048, 7)


7. Describe the characteristics of your qualitative variables by doing the following:

    * Determine the frequencies of each group within each categorical variable. Which groups have the highest frequency?
    * Determine the relative frequencies of each group within each categorical variable. Which groups have the highest relative frequencies?
    * Create a crosstab table using both the categorical variables in your dataset. How many mothers are smokers who have been pregnant before?

In [19]:
# parity - previously pregnant (yes/no)
# No previous pregnancy has the highest frequency.
dfq["parity"].value_counts()

No     766
Yes    282
Name: parity, dtype: int64

In [20]:
#smoke - smoking status of mother (nonsmoker/smoker)
# nonsmoker has the highest frequency.
dfq["smoke"].value_counts()

nonsmoker    634
smoker       414
Name: smoke, dtype: int64

In [21]:
dfq["parity"].value_counts(normalize = True)
# No previous pregnancy has the highest relative frequency with 73% of mothers having no previous prgnancy.

No     0.730916
Yes    0.269084
Name: parity, dtype: float64

In [22]:
dfq["smoke"].value_counts(normalize = True)
# nonsmoker has the highest relative frequency with 60.5% of mothers being nonsmokers.

nonsmoker    0.604962
smoker       0.395038
Name: smoke, dtype: float64

In [23]:
pd.crosstab(dfq["smoke"], dfq["parity"], margins = True)

parity      No  Yes   All
smoke                    
nonsmoker  462  172   634
smoker     304  110   414
All        766  282  1048

In [24]:
# 110 mothers are smokers who have been pregnant before.

8. Describe the Measures of Central Tendency (mean, median, mode) of the numeric variables in the dataset. Calculate the mean, median, and mode for all 5 numeric variables. For each median and mode, describe the following:

    * What does the median value tell you about the data?
    * What does the mode value tell you about the data?

In [25]:
for column in columns:
    print(f"{column} mean: {dfq[column].mean()}")
    print(f"{column} median: {dfq[column].median()}")
    print(f"{column} mode: {dfq[column].mode()[0]}")
    print(f"")

bwt mean: 120.18893129770993
bwt median: 120.0
bwt mode: 117

gestation mean: 280.0019083969466
gestation median: 280.0
gestation mode: 282.0

age mean: 27.170801526717558
age median: 26.0
age mode: 23.0

height mean: 64.03053435114504
height median: 64.0
height mode: 65.0

weight mean: 126.13454198473282
weight median: 125.0
weight mode: 130.0



In [26]:
# What does the median value tell you about the data?
# What does the mode value tell you about the data?

# bwt median: 120.0
# The median birth weight is 120 ounces.
# bwt mode: 117
# The most frequent value for weight is 117 ounces.

# gestation median: 280.0
# The median value for gestation is 280 weeks.
# gestation mode: 282.0
# The most frequent value for gestation is 282 weeks.

# age median: 26.0
# The median value for age of the mother is 26 years.
# age mode: 23.0
# The most frequent value for age of the mother is 23 weeks.
    
# height median: 64.0
# The median value for height of the mother is 64 inches.
# height mode: 65.0
# The most frequent value for height of the mother is 65 inches.
    
# weight median: 125.0
# The median value for weight of the mother is 125 pounds.
# weight mode: 130.0 
# The most frequent value for weight of the mother is 130 pounds.


9. Describe the Measures of Variability of the numeric variables in the dataset. Calculate the range, standard deviation, and 85th percentile for all 5 numeric variables. Answer the following questions:

    * Which column has the largest range? Which has the smallest? What does this mean?
    * Which column has the largest standard deviation? Which has the smallest? What does this tell you about those variables?

In [27]:
# range, standard deviation, and 85th percentile
for col in columns:
    col_range = dfq[col].max() - dfq[col].min()
    col_std = dfq[col].std()
    col_85_p = np.percentile(dfq[col],85)
    print(f"{col} range: {col_range} ")
    print(f"{col} std: {col_std } ")
    print(f"{col} 85th percentile: {col_85_p} ")
    print()

bwt range: 90 
bwt std: 16.44674168161553 
bwt 85th percentile: 137.0 

gestation range: 64.0 
gestation std: 11.371137417737582 
gestation 85th percentile: 292.0 

age range: 28.0 
age std: 5.721023623077813 
age 85th percentile: 34.0 

height range: 16.0 
height std: 2.4879888927471216 
height 85th percentile: 67.0 

weight range: 101.0 
weight std: 17.322549072820664 
weight 85th percentile: 145.0 



In [28]:
# Which column has the largest range? Which has the smallest? What does this mean?

# The weight column has the largest range of 101 lbs. The weight difference between 
# the lightest and heaviest mother is 101lbs.


# The heights column has the smallest range of 16 inches. The  height difference 
# between the shortest and tallest mother is 16 inches
